# MultiIndex
MultiIndex，即具有多个层次的索引，有些类似于根据索引进行分组的形式。通过多层次索引，我们就可以使用高层次的索引，来操作整个索引组的数据。  
## 创建方式
### 第一种
我们在创建Series或DataFrame时，可以通过给index（columns）参数传递多维数组，进而构建多维索引。【数组中每个维度对应位置的元素，组成每个索引值】  
多维索引的也可以设置名称（names属性），属性的值为一维数组，元素的个数需要与索引的层数相同（每层索引都需要具有一个名称）。
### 第二种
我们可以通过MultiIndex类的相关方法，预先创建一个MultiIndex对象，然后作为Series与DataFrame中的index（或columns）参数值。同时，可以通过names参数指定多层索引的名称。  
* from_arrays：接收一个多维数组参数，高维指定高层索引，低维指定底层索引。
* from_tuples：接收一个元组的列表，每个元组指定每个索引（高维索引，低维索引）。
* from_product：接收一个可迭代对象的列表，根据多个可迭代对象元素的笛卡尔积进行创建索引。

from_product相对于前两个方法而言，实现相对简单，但是，也存在局限。

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Series创建多层索引， 通过index指定一个多维的列表（数组）。
# 单层索引，指定一个一维数组。
# s = pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"])
# 多层索引，指定一个多维数组。多维数组中，逐级给出每层索引的值。
s = pd.Series([1, 2, 3, 4], index=[["A", "A", "B", "B"], ["a", "b", "c", "d"]])
# 多于多层索引，每一层都具有一个名字。
# s.index.names = ["abc", "def"]
# display(s)

In [3]:
# df = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=[["A", "A", "B"], ["a", 'b', 'b']])
# 很意外，此种方式不成功，我们需要使用numpy提供的数组生成函数才行。
# df = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], columns=[["A", "A", "B"], ["a", 'b', 'b']])
df = pd.DataFrame(np.arange(9).reshape(3, 3), columns=[["A", "A", "B"], ["a", 'b', 'b']])
display(df)
display(df["A"])
display(df["A", "a"])

A     B
   a  b  b
0  0  1  2
1  3  4  5
2  6  7  8

,a,b
0,0,1
1,3,4
2,6,7


0    0
1    3
2    6
Name: (A, a), dtype: int32

In [4]:
# 通过MultiIndex类的方法进行创建。

# 通过列表的方式进行创建。（每个内嵌列表元素指定层次的索引，[[第0层索引], [第1层索引]，……[第n层索引]]）
m = pd.MultiIndex.from_arrays([["A", "A", "B"], ["a", "b", "b"]])

# 通过元组构成列表的方式进行创建。[(高层，底层), (高层， 底层), ……]
m2 = pd.MultiIndex.from_tuples([("A", "a"), ("A", "b"), ("B", "b")])
df = pd.DataFrame(np.random.random((3, 3)), index=m2)

# 通过乘积（笛卡尔积）的方式进行创建。
m3 = pd.MultiIndex.from_product([["A", "B"], ["a", "b"]])
df = pd.DataFrame(np.random.random((4, 3)), index=m3)
df

0         1         2
A a  0.954184  0.052612  0.943119
  b  0.859722  0.267176  0.897029
B a  0.049726  0.901836  0.871435
  b  0.769453  0.475607  0.350911

## 多层索引操作
对于多层索引，同样也支持单层索引的相关操作，例如，索引元素，切片，索引数组选择元素等。我们也可以根据多级索引，按层次逐级选择元素。  
多层索引的优势：通过创建多层索引，我们就可以使用高层次的索引，来操作整个索引组的数据。  
格式：  
* s[操作]
* s.loc[操作]
* s.iloc[操作]

其中，操作可以是索引，切片，数组索引，布尔索引。  

### Series多层索引
* 通过loc（标签索引）操作，可以通过多层索引，获取该索引所对应的一组值。
* 通过iloc（位置索引）操作，会获取对应位置的元素值（与是否多层索引无关）。
* 通过s[操作]的行为有些诡异，建议不用。
 + 对于索引（单级），首先按照标签选择，如果标签不存在，则按照位置选择。
 + 对于多级索引，则按照标签进行选择。
 + 对于切片，如果提供的是整数，则按照位置选择，否则按照标签选择。
 + 对于数组索引, 如果数组元素都是整数，则根据位置进行索引，否则，根据标签进行索引。

In [5]:
s = pd.Series([1, 2, 3, 4], index=[["A", "A", "B", "B"], ["a", "b", "a", "b"]])
display(s)
# 通过访问外层索引，可以获取该外层索引对应的一组值。
# display(s["A"])
# 多层索引，不直接直接使用内层索引访问。
#  错误
# display(s["a"])
# 可以通过索引逐层进行访问
# display(s["A", "a"])

# 对于iloc，就是根据位置进行访问，与是否存在多层索引，关系不大。
# s.iloc[1]

s[["A", "B"]]

A  a    1
   b    2
B  a    3
   b    4
dtype: int64

A  a    1
   b    2
B  a    3
   b    4
dtype: int64

### DataFrame多层索引
* 通过loc（标签索引）操作，可以通过多层索引，获取该索引所对应的一组值。
* 通过iloc（位置索引）操作，会获取对应位置的一行（与是否多层索引无关）。
* 通过s[操作]的行为有些诡异，建议不用。
 + 对于索引，根据标签获取相应的列（如果是多层索引，则可以获得多列）。
 + 对于数组索引, 根据标签，获取相应的列（如果是多层索引，则可以获得多列）。
 + 对于切片，首先按照标签进行索引，然后再按照位置进行索引（取行）。

In [6]:
df = pd.DataFrame(np.random.rand(3, 3), index=[["A", "A", "B"], ["a", "b", "b"]])
# df.loc["A", "a"]
df.iloc[0]

0    0.284917
1    0.746716
2    0.575233
Name: (A, a), dtype: float64

## 交换索引
我们可以调用DataFrame对象的swaplevel方法来交换两个层级索引。该方法默认对倒数第2层与倒数第1层进行交换。我们也可以指定交换的层。层次从0开始，由外向内递增（或者由上到下递增），也可以指定负值，负值表示倒数第n层。除此之外，我们也可以使用层次索引的名称来进行交换。  

## 索引排序
我们可以使用sort_index方法对索引进行排序处理。
* level：指定根据哪一层进行排序，默认为最外（上）层。该值可以是数值，索引名，或者是由二者构成的列表。
* inplace：是否就地修改。默认为False。

In [7]:
df = pd.DataFrame(np.random.rand(4, 4), index=[["A", "A", "B", "B"], ["a1", "a1", "b1", "c1"], ["a2", "b2", "c2", "c2"]])
df.index.names = ["layer1", "layer2", "layer3"]
display(df)
# 多层索引，编号从外向内，0， 1， 2， 3.同时，索引编号也支持负值。
# 负值表示从内向外，-1， -2， -3. -1表示最内容。

# 交换多层索引的两层。默认交换最内层与倒数第二层。（最里面的两层）
# df.swaplevel(0, 2)
# 交换多层索引时，我们除了可以指定层次的编号外，也可以指定索引层次的名称。
df.swaplevel("layer1", "layer3")

0         1         2         3
layer1 layer2 layer3                                        
A      a1     a2      0.984698  0.615676  0.753877  0.932709
              b2      0.493305  0.262877  0.542709  0.687143
B      b1     c2      0.244724  0.528395  0.653433  0.178914
       c1     c2      0.316843  0.366835  0.612388  0.804049

0         1         2         3
layer3 layer2 layer1                                        
a2     a1     A       0.984698  0.615676  0.753877  0.932709
b2     a1     A       0.493305  0.262877  0.542709  0.687143
c2     b1     B       0.244724  0.528395  0.653433  0.178914
       c1     B       0.316843  0.366835  0.612388  0.804049

In [8]:
df = pd.DataFrame(np.random.rand(4, 4), index=[["A", "B", "B", "A"], ["b", "b", "a", "c"], ["b2", "c2", "a2", "c2"]])
df.index.names = ["layer1", "layer2", "layer3"]
display(df)
# 索引排序，默认进行最外层的排序。
# df.sort_index()
# 自定义排序的层次。
df.sort_index(level=1)
# 也可以通过索引的名称进行排序。
df.sort_index(level="layer1")

0         1         2         3
layer1 layer2 layer3                                        
A      b      b2      0.648595  0.009719  0.251170  0.935658
B      b      c2      0.038701  0.628436  0.313889  0.085547
       a      a2      0.257360  0.822571  0.070936  0.719467
A      c      c2      0.817126  0.693073  0.309432  0.837334

0         1         2         3
layer1 layer2 layer3                                        
A      b      b2      0.648595  0.009719  0.251170  0.935658
       c      c2      0.817126  0.693073  0.309432  0.837334
B      a      a2      0.257360  0.822571  0.070936  0.719467
       b      c2      0.038701  0.628436  0.313889  0.085547

## 索引堆叠
通过DataFrame对象的stack方法，可以进行索引堆叠，即将指定层级的列转换成行。
level：指定转换的层级，默认为-1。
## 取消堆叠
通过DataFrame对象的unstack方法，可以取消索引堆叠，即将指定层级的行转换成列。
level：指定转换的层级，默认为-1。
fill_value：指定填充值。默认为NaN。

In [9]:
df = pd.DataFrame(np.random.rand(4, 4), index=[["A", "B", "B", "A"], ["b", "b", "a", "c"], ["b2", "c2", "a2", "c2"]])
df.index.names = ["layer1", "layer2", "layer3"]
display(df)
# 取消堆叠，如果没有匹配的数据，则显示空值NaN。
# df.unstack()
# 我们可以指定值去填充NaN（空值）。
# df.unstack(fill_value=50000)
# unstack默认会将最内层取消堆叠，我们也可以自行来指定层次。
# df.unstack(0)
# 除了指定层次序号外，我们也可以指定层次（索引）的名称。
# df.unstack("layer1")

# 进行堆叠  列-》行      取消堆叠   行-》列
df.stack()
# stack堆叠也可以指定层次。
df.stack(0)
# stack堆叠也可以通过索引名进行操作。

0         1         2         3
layer1 layer2 layer3                                        
A      b      b2      0.630500  0.974254  0.963365  0.618260
B      b      c2      0.547420  0.756999  0.149537  0.756407
       a      a2      0.115701  0.476125  0.179361  0.585364
A      c      c2      0.684255  0.874848  0.555394  0.790719

layer1  layer2  layer3   
A       b       b2      0    0.630500
                        1    0.974254
                        2    0.963365
                        3    0.618260
B       b       c2      0    0.547420
                        1    0.756999
                        2    0.149537
                        3    0.756407
        a       a2      0    0.115701
                        1    0.476125
                        2    0.179361
                        3    0.585364
A       c       c2      0    0.684255
                        1    0.874848
                        2    0.555394
                        3    0.790719
dtype: float64

## 设置索引
在DataFrame中，如果我们需要将现有的某一（几）列作为索引列，可以调用set_index方法来实现。
* drop：是否丢弃作为新索引的列，默认为True。
* append：是否以追加的方式设置索引，默认为False。
* inplace：是否就地修改，默认为False。

## 重置索引
调用在DataFrame对象的reset_index，可以重置索引。该操作与set_index正好相反。
* level：重置索引的层级，默认重置所有层级的索引。如果重置所有索引，将会创建默认整数序列索引。
* drop：是否丢弃重置的索引列，默认为False。
* inplace：是否就地修改，默认为False。

In [10]:
df = pd.DataFrame({"pk":[1, 2, 3, 4], "name":["n1", "n2", "n3", "n4"], "age":[15, 20, 17, 8]})
# 字典是无序的，如果需要顺序调整。
# df = df[["pk", "name", "age"]]
# 也可以通过reindex来重新组织索引。
# df.reindex(["pk", "name", "age"], axis=1)

# 设置参数指定的列，充当新的索引。
# df.set_index("pk")
# 也可以设置层级索引。
# df.set_index(["pk", "name"])

# 默认情况下，充当新索引的列会被丢弃，我们可以通过drop=False设置依然保留。
# df = df.set_index("pk", drop=False)
# append用来设置是否以追加的形式设置索引（层级索引）。默认为False（取代以前的索引）。
# df.set_index("name", append=True)


# df.set_index("pk", inplace=True)
# 重置索引。默认情况下重置所有层次的索引。如果所有层次的索引均被重置，则重新生成整数序列的索引。
# df.reset_index()

df.set_index(["pk", "name"], inplace=True)
# 我们也可以指定重置索引的层次。【如果重置索引之后，还有索引层次（没有重置所有层次的索引），则不会重新生成整数序列的索引。】
# df.reset_index(0, inplace=True)

# 重置索引后，默认重置索引会充当新的列，如果不需要重置的索引充当新的列，可以指定drop=True。
df.reset_index(0, drop=True)

,age
name,
n1,15
n2,20
n3,17
n4,8


# 分组与聚合
分组与聚合操作与数据库中的分组与聚合相似。  
## groupby分组
我们可以通过groupby方法来对Series或DataFrame对象实现分组操作。该方法会返回一个分组对象。不过，如果直接查看（输出）该对象，并不能看到任何的分组信息。
* groups（属性）：返回一个字典类型对象，包含分组信息。
* size：返回每组记录的数量。
* discribe：分组查看统计信息。

## 迭代
我们也可以使用for循环来对分组对象进行迭代。迭代每次会返回一个元组，第1个元素为用来分组的key，第2个元素为该组对应的数据。

## 分组的方式
使用groupby进行分组时，分组的参数可以是如下的形式：
* 索引名：根据该索引进行分组。
* 索引名构成的数组：根据数组中每个索引进行分组。
* 字典或Series：key指定索引，value指定分组依据，即value值相等的，会分为一组。
* 函数：接受索引，返回分组依据的value值。

## apply
对分组对象，可以调用apply函数，该函数接收每个组的数据，返回操作之后的结果。apply最后会将每个组的操作结果进行合并（concat）。

In [11]:
df = pd.DataFrame({"部门":["A", "A", "B", "B"], "利润":[10, 20, 15, 28], "人员":["a", "b", "c", "d"], "年龄":[20, 15, 18, 30]})
# 根据部门进行分组，返回一个分组对象。
# display(df)
g = df.groupby("部门")
# 返回分组对象的信息。
# display(g.groups)
# 返回每组的数量。
# display(g.size())
# 返回每组的统计信息。
# display(g.describe())

# 分组信息不像列表那样，我们可以直接输出查看。（类似于迭代器的机制，是需要时，动态进行计算的。）
# 我们可以通过for循环来查看分组对象的数据。类似字典的形式，key-> 分组列的值。 value-》分组对应的数据记录。
# for k, v in g:
#     display(k, v)
    
# display(df)
# 在分组上进行聚合统计。

# 在整个DataFrame上进行聚合。
# g.sum()
# 在指定的列上进行聚合。
# g["利润"].sum()

# display(g)
# display(g["利润"])

In [ ]:
df = pd.DataFrame({"部门":["A", "A", "B", "B"],"小组":["g1", "g2", "g1", "g2"], "利润":[10, 20, 15, 28], "人员":["a", "b", "c", "d"], "年龄":[20, 15, 18, 30]})
display(df)
# 分组的方式
# 1 进行单一的分组，提供单一的标签。
# df.groupby("部门")
# 2 进行多分组，提供多标签构成的列表。
# g = df.groupby(["部门", "小组"])

# display(g)
# i = g.__iter__()
# i = iter(g)
# i.__next__()
# next(i)
# 3 提供一个字典(Series)，key提供索引， value提供组名。结果就会根据value相同的记录，分到一组。
# g = df.groupby({0:1, 1:1, 2:2, 3:2})
# 我们还可以根据列进行分组。将axis参数指定为1。
# g = df.groupby({"人员":1, "利润":1, "小组":2, "年龄":2, "部门":2}, axis=1)

# 4 提供一个函数。函数需要具有一个参数。用来依次接收索引值。函数还需要具有一个返回值，用来指定组。
# 如果看成字典的形式，函数接收key，返回value。
g = df.groupby(lambda x: x % 2)

# for k, v in g:
#     display(k, v)
# display(g.sum())

# DataFrame 进行聚合时,默认会包含所有列，我们可以指定numeric_only参数为True，这样只会聚合（统计）数值类型的行（列）。
# display(df.sum(numeric_only=True))

In [12]:
# 分组的方式：
# 每个索引，会依次传递到函数中（index）。
def group_helper(index):
#     if index > 5:
#         return 0
#     else:
#         return 1
    return 0 if index > 2 else 1
# 对于传递进行的索引 0， 1， 2， 3
# 每个索引对应的返回值
# 0 -> 1
# 1 -> 1
# 2 -> 1
# 3 -> 0
# 根据函数返回的value值，索引0， 1, 2 分到一组，索引3分到一组。

# 类似的，对于Series或字典来说，
# df.groupby({0:1, 1:1, 2:1, 3:0})
    
# df.groupby(group_helper)

In [ ]:
# 分组对象也可以进行apply。apply接收一个函数（参数）。函数具有一个参数，用来依次接收每一个分组数据，并且返回每一个分组数据的结果。
df = pd.DataFrame({"部门":["A", "A", "B", "B"],"小组":["g1", "g2", "g1", "g2"], "利润":[10, 20, 15, 28], 
                   "人员":["a", "b", "c", "d"], "年龄":[20, 15, 18, 30]})
display(df)
g = df.groupby(lambda x: x % 2)
# apply当前实现的细节：对于第一个分组，会调用函数两次。（但是，不影响apply最后进行concat合并的结果）
# g.apply(lambda x: x.sum(numeric_only=True))
# g.apply(lambda x: x.max(numeric_only=True) - x.min(numeric_only=True))
g.apply(lambda x: x.describe().T)

# display(g.describe())

## 聚合
可以在分组对象上进行聚合（多个值变成一个值）。例如，mean()，sum()等。  
除此之外，我们也可以调用agg方法，实现自定义的聚合方式。函数接收一行（一列），返回该行（列）聚合后的结果。

In [ ]:
df = pd.DataFrame({"利润":[10, 20, 15, 28], "年龄":[20, 15, 18, 30]})
display(df)
# DataFrame与分组对象可以调用agg / aggregate方法，来实现聚合操作。

# 关于agg方法，可以接收的参数类型：
# 1 函数说明字符串，例如，sum，mean，std
# display(df.mean())
# display(df.agg("mean"))
# 2 多个函数字符串构成的列表
# display(df.agg(["mean", "sum", "max"]))
# 3 字典 key：指定列标签，value：指定统计的信息。
# 优势：可以针对不同的列， 提供不同的聚合信息。
# display(df.agg({"利润":["mean", "sum"] ,  "年龄":["max", "min"]}))
# 4 函数, 可以实现自己的聚合方式。函数具有一个参数，用来接收DataFrame传递过来的每一列（行），返回操作之后的结果。
# df.agg(lambda x: x.mean())

# 分组对象的agg，与DataFrame相似。
g = df.groupby(lambda x: x % 2)
# g.mean()
# g.agg("mean")
# g.agg(["mean", "sum"])
# display(g.agg({"利润":["mean", "sum"] ,  "年龄":["max", "min"]}))
# 函数接收的是每个组的每一个列（行）。函数的返回值作为最终聚合的结果。
g.agg(lambda x: x.max())

# 练习
按照天来统计电影信息的票房信息。（票房总数，平均值，最大值，最小值，电影数量）

In [ ]:
df = pd.read_csv("spider.csv", header=None)
# 首先，进行数据的过滤，=》 只保留电影的信息记录
t = df[df[1].str.startswith("http://www.movie.com/bor/")]
# 3, 4两列对我们没有任何帮助，可以删除。
t.drop([3, 4], axis=1, inplace=True)
# 查看电影信息是否存在空值。是否存在重复的数据，删除。
# 通过info查看，没有空值。
# t.info()
# 查看重复的记录。
# t[t.duplicated()]
# 删除重复的记录。
t.drop_duplicates(inplace=True)
# 针对第2列，拆分,并扩展。
sp = t[2].str.split(";", expand=True)
# 删除没有用的8,9列。
sp.drop([8, 9], axis=1, inplace=True)
# 对于第7列（票房）列进行调整。提取其中的数值信息。注意：需要进行类型转换
sp[7] = sp[7].str.replace("票房（万）", "").astype(np.float64)
# 正则表达式的提取操作。
# sp[7] = sp[7].str.extract(r"([0-9.]+)")
# 与原有的DataFrame合并。[concat  merge   join]
result = pd.concat((t, sp), axis=1)
# result.columns.size
# len(result.columns)
# 硬编码，不要使用。
# result.columns = list(range(11))
result.columns = list(range(result.columns.size))
# 进行groupby时，不能含有索引名称相同的列。否则会产生错误。
g = result.groupby(0)
# g[10].sum().sort_values(ascending=False)
g.agg(["sum", "mean", "max", "min", "count"])